In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [2]:
# Importa e inicia o findspark
import findspark
findspark.init()

In [3]:
# Importa todos os módulos necessários
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [4]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[7]") # usa a márquina local com 7 nucleos
                     .appName("Aceleração PySpark - Capgemini"))

In [5]:
# Cria o Schema de todos os datasets
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField('tailnum', StringType(), True),
    StructField('year', IntegerType(), True),
    StructField('type', StringType(), True),
    StructField('manufacturer', StringType(), True),
    StructField('model', StringType(), True),
    StructField('engines', IntegerType(), True),
    StructField('seats', IntegerType(), True),
    StructField('speed', IntegerType(), True),
    StructField('engine', StringType(), True)
])

schema_flights = StructType([
    StructField('year', IntegerType(), True),
    StructField('month', IntegerType(), True),
    StructField('day', IntegerType(), True),
    StructField('dep_time', StringType(), True),
    StructField('dep_delay', IntegerType(), True),
    StructField('arr_time', StringType(), True),
    StructField('arr_delay', IntegerType(), True),
    StructField('carrier', StringType(), True),
    StructField('tailnum', StringType(), True),
    StructField('flight', StringType(), True),
    StructField('origin', StringType(), True),
    StructField('dest', StringType(), True),
    StructField('air_time', IntegerType(), True),
    StructField('distance', IntegerType(), True),
    StructField('hour', IntegerType(), True),
    StructField('minute', IntegerType(), True)
])

In [6]:
# Cria os dataframes
df_airports = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("../data/airports.csv"))

df_planes = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("../data/planes.csv"))

df_flights = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("../data/flights.csv"))

In [7]:
df_airports.show(3)
df_planes.show(3)
df_flights.show(3)

+---+--------------------+---------+---------+----+---+---+
|faa|                name|      lat|      lon| alt| tz|dst|
+---+--------------------+---------+---------+----+---+---+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|
+---+--------------------+---------+---------+----+---+---+
only showing top 3 rows

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
+--

In [ ]:
# Função de regex
def regex_definition(element):
    return r'|'.join(map(lambda x : f'.*{x}*', element))

# -----------------------------------------------------------------------------------------------------------
# Transformação airports
# -----------------------------------------------------------------------------------------------------------
## 1.

In [ ]:
df_airports.filter(F.col('alt') < 0).show()

In [ ]:
df_airports = df_airports.withColumn('alt', 
                                     F.when(F.col('alt') < 0, 0)
                                      .otherwise(F.col('alt'))
                                    )

In [ ]:
df_airports.filter(F.col('alt') == 0).count()

In [ ]:
df_airports.groupBy(F.col('alt')).count().orderBy(F.col('alt')).show()

# -----------------------------------------------------------------------------------------------------------
## 2.

In [ ]:
df_airports.filter(F.col('tz').between(-7, -5)).show(10)

In [ ]:
df_airports = df_airports.withColumn('dst',
                                     F.when(F.col('tz').between(-7, -5), 'A')
                                      .otherwise(F.col('dst'))
                                    )

In [ ]:
df_airports.filter(F.col('tz').between(-7, -5)).show(10)
df_airports.groupBy("dst").count().show()

In [ ]:
df_airports.groupBy(F.col('dst')).count().show(999)

# -----------------------------------------------------------------------------------------------------------
## 3.

In [ ]:
df_airports.filter(F.col('dst') == 'U').show()

In [ ]:
df_airports = df_airports.withColumn('dst',
                                     F.when(F.col('dst') == 'U', 'A')
                                      .otherwise(F.col('dst'))
                                    )

In [ ]:
df_airports.filter(F.col('faa') == 'GCW').show()
df_airports.filter(F.col('dst') == 'A').count()
df_airports.groupBy("dst").count().show()

# -----------------------------------------------------------------------------------------------------------
## 4.

In [ ]:
# Alaska
df_airports.filter(F.col('lon') < -124).show(5)

# Mainland-west
df_airports.filter(F.col('lon').between(-124, -95)).show(5)

# Mainland-east
df_airports.filter(F.col('lon').between(-95, -59)).show(5)

# Offshore
df_airports.filter((F.col('lon') > -50) | (F.col('lat') < 24)).show(5)

In [ ]:
df_airports = df_airports.withColumn('region',
                                      F.when(F.col('lon') < -124,                        'ALASKA')
                                       .when(F.col('lon').between(-124, -95),            'MAINLAND-WEST')
                                       .when(F.col('lon').between(-95, -54),             'MAINLAND-EAST')
                                       .when((F.col('lon') > -50) | (F.col('lat') > 24), 'OFFSHORE')
                                       .otherwise('NaN')
                                    )

df_airports.show(10)

In [ ]:
df_airports.groupBy("region").count().show()

# -----------------------------------------------------------------------------------------------------------
## 5.

In [ ]:
LIST_AP  = ['Airport', 'Tradeport', 'Heliport', 'Airpor', 'Arpt']
REGEX_AP = regex_definition(LIST_AP)

LIST_AD  = ['Aerodrome']
REGEX_AD = regex_definition(LIST_AD)

LIST_AK  = ['Airpark', 'Aero Park']
REGEX_AK = regex_definition(LIST_AK)

LIST_AS  = ['Station', 'Air Station']
REGEX_AS = regex_definition(LIST_AS)

LIST_FL  = ['Field', 'Fld']
REGEX_FL = regex_definition(LIST_FL)

In [ ]:
df_airports = df_airports.withColumn('type',
                                     F.when(F.col('name').rlike(REGEX_AP),  'AP')
                                       .when(F.col('name').rlike(REGEX_AD), 'AD')
                                       .when(F.col('name').rlike(REGEX_AK), 'AK')
                                       .when(F.col('name').rlike(REGEX_AS), 'AS')
                                       .when(F.col('name').rlike(REGEX_FL), 'FL')
                                       .otherwise('NaN')
                                    )

In [ ]:
df_airports.filter((F.col('name').rlike('(Airport).*(Field)'))).show(10, truncate=False)
df_airports.groupBy("type").count().show()

# -----------------------------------------------------------------------------------------------------------
## 6.

In [ ]:
df_airports.filter(F.col('name').rlike('Afs')).show(10, False)

In [ ]:
LIST_MILITARY  = ['Base', 'Aaf', 'Afs', 'Ahp', 'Afb', 'LRRS', 'Lrrs', 'Arb', 'Naf', 'NAS', 'Nas', 'Jrb', 'Ns', 'As', 'Cgas', 'Angb']
REGEX_MILITARY = r'|'.join(map(lambda x : f'^{x} | {x} | {x}$', LIST_MILITARY))

In [ ]:
df_airports = df_airports.withColumn('military',
                                     F.when(F.col('name').rlike(REGEX_MILITARY), True)
                                      .otherwise(False)
                                    )

In [ ]:
df_airports.filter(F.col('military') == True).show(10, False)
df_airports.filter(F.col('name').rlike('As')).show(50, False)
df_airports.groupBy("military").count().show()

# -----------------------------------------------------------------------------------------------------------
## 7.

In [ ]:
df_airports.filter(F.col('name').rlike('(City).*(Intl)')).show(10, False)

In [ ]:
LIST_I  = ['International', 'Intl', 'Intercontinental']
REGEX_I = regex_definition(LIST_I)

LIST_N  = ['National', 'Natl']
REGEX_N = regex_definition(LIST_N)

LIST_R  = ['Regional', 'Reigonal', 'Rgnl', 'County', 'Metro', 'Metropolitan']
REGEX_R = regex_definition(LIST_R)

LIST_M  = ['Municipal', 'Muni', 'City']
REGEX_M = regex_definition(LIST_M)

In [ ]:
df_airports = df_airports.withColumn('administration',
                                     F.when(F.col('name').rlike(REGEX_I), 'I')
                                      .when(F.col('name').rlike(REGEX_N), 'N')
                                      .when(F.col('name').rlike(REGEX_R), 'R')
                                      .when(F.col('name').rlike(REGEX_M), 'M')
                                      .otherwise('NaN')
                                    )

df_airports.show(10, False)

In [ ]:
df_airports.filter(F.col('administration') == 'I').show(10, False)
df_airports.groupBy("administration").count().show()

# -----------------------------------------------------------------------------------------------------------
# Transformação planes
# -----------------------------------------------------------------------------------------------------------
## 1.

In [ ]:
df_planes = df_planes.withColumn('tailchar', F.regexp_replace(F.col('tailnum'), '[0-9]|^N', ""))

df_planes.show()

In [ ]:
df_planes.groupBy("tailchar").count().show()

# -----------------------------------------------------------------------------------------------------------
## 2.

In [ ]:
df_planes.filter(F.col('year') == 0).show()

In [ ]:
df_planes = df_planes.withColumn('year',
                                 F.when(F.col('year') == 0, 1996)
                                  .otherwise(F.col('year')))
 
df_planes.filter(F.col('year') == 0).show()
df_planes.groupBy("year").count().show()

# -----------------------------------------------------------------------------------------------------------
## 3.

In [ ]:
df_planes.filter(F.col('year').isNull()).show(10)
df_planes.filter(F.col('year').isNull()).count()

In [ ]:
df_planes_order = df_planes.orderBy(F.col('manufacturer'), F.col('model'), F.col('year')).filter(F.col('year').isNotNull())
df_planes_order.show(10)

In [ ]:
df_planes.filter(F.col('manufacturer') == 'LAMBERT RICHARD').show()

In [ ]:
# Reduz o dataframe
df_yeara = df_planes_order.groupBy("manufacturer", "model").min("year").orderBy(F.col('manufacturer'), F.col('model'))
df_yeara.show()

In [ ]:
# Reduz ainda mais
df_yearb = df_yeara.groupBy('manufacturer').min('min(year)').orderBy(F.col('manufacturer'))
df_yearb.show()

In [ ]:
# Renomeando as colunar do dataframe year_a e de year_b
df_yeara = (df_yeara.withColumnRenamed('manufacturer', 'manuf_a')
                    .withColumnRenamed('model', 'model_a')
                    .withColumnRenamed('min(year)', 'year_a'))

df_yearb = (df_yearb.withColumnRenamed('manufacturer', 'manuf_b')
                    .withColumnRenamed('min(min(year))', 'year_b'))

In [ ]:
# Unindo os dataframe df_planes + df_yeara + df_yearb 
df_join = df_planes.join(df_yeara, 
                        (df_planes['manufacturer'] == df_yeara['manuf_a']) &
                        (df_planes['model'] == df_yeara['model_a']),
                        'left')

df_join = df_join.join(df_yearb,
                       df_join['manufacturer'] == df_yearb['manuf_b'],
                       'left')

In [ ]:
# Verificando se a união ocorreu corretamente
df_join.show(3, 7)
df_join.filter(F.col('year').isNull()).show(20, 7)

In [ ]:
# aplicando as mudanças na coluna year
df_join = df_join.withColumn('year',
                              F.when((F.col('year').isNull()) &
                                     (F.col('manuf_a').isNotNull()) &
                                     (F.col('model_a').isNotNull()), F.col('year_a'))
                               .when((F.col('year').isNull()) &
                                     (F.col('manuf_b').isNotNull()), F.col('year_b'))
                               .otherwise(F.col('year'))
                            )

df_join.filter(F.col('year').isNull()).show(truncate = 7)

In [ ]:
df_planes = df_join.drop('manuf_a', 
                         'model_a', 
                         'year_a', 
                         'manuf_b',
                         'year_b')

In [ ]:
df_planes.filter(F.col('year').isNull()).show()